In [1]:
import os
import sys
from functools import partial
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import joblib
%matplotlib inline
import seaborn as sns
import multiprocessing as mp

sys.path.append('../')
#from src.utils import parallel_apply
#from src.feature_extraction import add_features_in_group

warnings.filterwarnings('ignore')

DIR = '/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data/'
#description = pd.read_csv(os.path.join(DIR,'HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv(os.path.join(DIR, 'application_test.csv'))
bureau = pd.read_csv(os.path.join(DIR, 'bureau.csv'))



In [4]:
bureau['bureau_credit_active_binary'] = (bureau['CREDIT_ACTIVE'] != 'Closed').astype(int) #active=1, closed=0
bureau['bureau_credit_enddate_binary'] = (bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int) #not end=1, end=0
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,bureau_credit_active_binary,bureau_credit_enddate_binary
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN,0,0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN,1,1
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,1,1
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,1,0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN,1,0
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN,0,0
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN,0,0
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN,0,0


In [5]:
features = pd.DataFrame({'SK_ID_CURR':bureau['SK_ID_CURR'].unique()})
features

,SK_ID_CURR
0,215354
1,162297
2,402440
3,238881
4,222183
...,...
305806,207190
305807,324956
305808,448157
305809,345866


In [6]:
groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])

# 1.bureau_number_of_past_loans
group_object = groupby_SK_ID_CURR['DAYS_CREDIT'].agg('count').reset_index()
group_object.rename(index=str, columns={'DAYS_CREDIT': 'bureau_number_of_past_loans'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,bureau_number_of_past_loans
0,215354,11
1,162297,6
2,402440,1
3,238881,8
4,222183,8


In [7]:
#2. bureau_number_of_loan_types
group_object = groupby_SK_ID_CURR['CREDIT_TYPE'].agg('nunique').reset_index()
group_object.rename(index=str, columns={'CREDIT_TYPE': 'bureau_number_of_loan_types'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types
0,215354,11,3
1,162297,6,3
2,402440,1,1
3,238881,8,2
4,222183,8,3


In [8]:
# 3. bureau_average_of_past_loans_per_type
features['bureau_average_of_past_loans_per_type'] = \
    features['bureau_number_of_past_loans'] / features['bureau_number_of_loan_types']
    
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type
0,215354,11,3,3.666667
1,162297,6,3,2.000000
2,402440,1,1,1.000000
3,238881,8,2,4.000000
4,222183,8,3,2.666667


In [9]:
# 4. bureau_credit_active_percen
group_object = groupby_SK_ID_CURR['bureau_credit_active_binary'].agg('mean').reset_index()
group_object.rename(index=str, columns={'bureau_credit_active_binary': 'bureau_credit_active_percen'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen
0,215354,11,3,3.666667,0.545455
1,162297,6,3,2.000000,0.500000
2,402440,1,1,1.000000,1.000000
3,238881,8,2,4.000000,0.375000
4,222183,8,3,2.666667,0.625000


In [10]:
# 5. bureau_total_customer_debt
group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM_DEBT'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_CREDIT_SUM_DEBT': 'bureau_total_customer_debt'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt
0,215354,11,3,3.666667,0.545455,284463.18
1,162297,6,3,2.000000,0.500000,0.00
2,402440,1,1,1.000000,1.000000,76905.00
3,238881,8,2,4.000000,0.375000,552730.50
4,222183,8,3,2.666667,0.625000,1185081.84


In [11]:
#6. bureau_total_customer_credit
group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_CREDIT_SUM': 'bureau_total_customer_credit'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30
1,162297,6,3,2.000000,0.500000,0.00,8230386.15
2,402440,1,1,1.000000,1.000000,76905.00,89910.00
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00


In [12]:
# 7. bureau_debt_credit_ratio
features['bureau_debt_credit_ratio'] = \
    features['bureau_total_customer_debt'] / features['bureau_total_customer_credit']
    
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538


In [13]:
# 8. bureau_total_customer_overdue
group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM_OVERDUE'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_CREDIT_SUM_OVERDUE': 'bureau_total_customer_overdue'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio,bureau_total_customer_overdue
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617,0.0
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000,0.0
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355,0.0
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060,0.0
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538,0.0


In [14]:
# 9. bureau_overdue_debt_ratio
features['bureau_overdue_debt_ratio'] = \
    features['bureau_total_customer_overdue'] / features['bureau_total_customer_debt']
    
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio,bureau_total_customer_overdue,bureau_overdue_debt_ratio
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617,0.0,0.0
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000,0.0,NaN
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355,0.0,0.0
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060,0.0,0.0
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538,0.0,0.0


In [15]:
# 10. bureau_total_prolonged_count
group_object = groupby_SK_ID_CURR['CNT_CREDIT_PROLONG'].agg('sum').reset_index()
group_object.rename(index=str, columns={'CNT_CREDIT_PROLONG': 'bureau_total_prolonged_count'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio,bureau_total_customer_overdue,bureau_overdue_debt_ratio,bureau_total_prolonged_count
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617,0.0,0.0,0
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000,0.0,NaN,0
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355,0.0,0.0,0
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060,0.0,0.0,0
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538,0.0,0.0,0


In [16]:
#11. bureau_credit_enddate_percentage
group_object = groupby_SK_ID_CURR['bureau_credit_enddate_binary'].agg('mean').reset_index()
group_object.rename(index=str, columns={'bureau_credit_enddate_binary': 'bureau_credit_enddate_percentage'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio,bureau_total_customer_overdue,bureau_overdue_debt_ratio,bureau_total_prolonged_count,bureau_credit_enddate_percentage
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617,0.0,0.0,0,0.454545
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000,0.0,NaN,0,0.166667
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355,0.0,0.0,0,1.000000
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060,0.0,0.0,0,0.125000
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538,0.0,0.0,0,0.625000


In [17]:
features

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio,bureau_total_customer_overdue,bureau_overdue_debt_ratio,bureau_total_prolonged_count,bureau_credit_enddate_percentage
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617,0.0,0.0,0,0.454545
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000,0.0,NaN,0,0.166667
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355,0.0,0.0,0,1.000000
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060,0.0,0.0,0,0.125000
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538,0.0,0.0,0,0.625000
...,...,...,...,...,...,...,...,...,...,...,...,...
305806,207190,1,1,1.000000,0.000000,0.00,450000.00,0.000000,0.0,NaN,0,1.000000
305807,324956,1,1,1.000000,0.000000,0.00,19800.00,0.000000,0.0,NaN,0,0.000000
305808,448157,1,1,1.000000,1.000000,1076809.50,1800000.00,0.598228,0.0,0.0,0,1.000000
305809,345866,1,1,1.000000,0.000000,0.00,175054.50,0.000000,0.0,NaN,0,0.000000


In [18]:
bureau['bureau_credit_type_consumer'] = (bureau['CREDIT_TYPE'] == 'Consumer credit').astype(int)
bureau['bureau_credit_type_car'] = (bureau['CREDIT_TYPE'] == 'Car loan').astype(int)
bureau['bureau_credit_type_mortgage'] = (bureau['CREDIT_TYPE'] == 'Mortgage').astype(int)
bureau['bureau_credit_type_credit_card'] = (bureau['CREDIT_TYPE'] == 'Credit card').astype(int)
bureau['bureau_credit_type_other'] = (~(bureau['CREDIT_TYPE'].isin(['Consumer credit',
                                                        'Car loan', 'Mortgage', 'Credit card']))).astype(int)
bureau['bureau_unusual_currency'] = (~(bureau['CREDIT_CURRENCY'] == 'currency 1')).astype(int)

In [19]:
groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])
group_object = groupby_SK_ID_CURR[['bureau_credit_type_consumer',
                                   'bureau_credit_type_car',
                                   'bureau_credit_type_mortgage',
                                   'bureau_credit_type_credit_card',
                                   'bureau_credit_type_other',
                                   'bureau_unusual_currency']].agg(['sum', 'mean']).reset_index()

group_object.columns = ['{}_{}'.format(a,b) for a,b in group_object.columns.tolist()]
group_object.rename(index=str, columns={'SK_ID_CURR_': 'SK_ID_CURR'},inplace=True)
group_object

,SK_ID_CURR,bureau_credit_type_consumer_sum,bureau_credit_type_consumer_mean,bureau_credit_type_car_sum,bureau_credit_type_car_mean,bureau_credit_type_mortgage_sum,bureau_credit_type_mortgage_mean,bureau_credit_type_credit_card_sum,bureau_credit_type_credit_card_mean,bureau_credit_type_other_sum,bureau_credit_type_other_mean,bureau_unusual_currency_sum,bureau_unusual_currency_mean
0,100001,7,1.000000,0,0.0,0,0.0,0,0.000000,0,0.000000,0,0.0
1,100002,4,0.500000,0,0.0,0,0.0,4,0.500000,0,0.000000,0,0.0
2,100003,2,0.500000,0,0.0,0,0.0,2,0.500000,0,0.000000,0,0.0
3,100004,2,1.000000,0,0.0,0,0.0,0,0.000000,0,0.000000,0,0.0
4,100005,2,0.666667,0,0.0,0,0.0,1,0.333333,0,0.000000,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,456249,9,0.692308,0,0.0,0,0.0,3,0.230769,1,0.076923,0,0.0
305807,456250,2,0.666667,0,0.0,0,0.0,1,0.333333,0,0.000000,0,0.0
305808,456253,3,0.750000,0,0.0,0,0.0,1,0.250000,0,0.000000,0,0.0
305809,456254,1,1.000000,0,0.0,0,0.0,0,0.000000,0,0.000000,0,0.0


In [20]:
features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio,bureau_total_customer_overdue,bureau_overdue_debt_ratio,...,bureau_credit_type_car_sum,bureau_credit_type_car_mean,bureau_credit_type_mortgage_sum,bureau_credit_type_mortgage_mean,bureau_credit_type_credit_card_sum,bureau_credit_type_credit_card_mean,bureau_credit_type_other_sum,bureau_credit_type_other_mean,bureau_unusual_currency_sum,bureau_unusual_currency_mean
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617,0.0,0.0,...,1,0.090909,0,0.000000,3,0.272727,0,0.0,0,0.0
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000,0.0,NaN,...,0,0.000000,1,0.166667,2,0.333333,0,0.0,0,0.0
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355,0.0,0.0,...,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060,0.0,0.0,...,0,0.000000,0,0.000000,3,0.375000,0,0.0,0,0.0
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538,0.0,0.0,...,1,0.125000,0,0.000000,3,0.375000,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,207190,1,1,1.000000,0.000000,0.00,450000.00,0.000000,0.0,NaN,...,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0
305807,324956,1,1,1.000000,0.000000,0.00,19800.00,0.000000,0.0,NaN,...,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0
305808,448157,1,1,1.000000,1.000000,1076809.50,1800000.00,0.598228,0.0,0.0,...,0,0.000000,1,1.000000,0,0.000000,0,0.0,0,0.0
305809,345866,1,1,1.000000,0.000000,0.00,175054.50,0.000000,0.0,NaN,...,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0


In [21]:
bureau.sort_values(['SK_ID_CURR','DAYS_CREDIT'],ascending=False, inplace=True)
bureau['days_credit_diff'] = bureau['DAYS_CREDIT'].diff().replace(np.nan, 0)    

groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])

In [22]:
group_object = groupby_SK_ID_CURR['bureau_credit_active_binary'].agg('sum').reset_index()
group_object.rename(index=str, columns={'bureau_credit_active_count': 'bureau_credit_active_binary'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio,bureau_total_customer_overdue,bureau_overdue_debt_ratio,...,bureau_credit_type_car_mean,bureau_credit_type_mortgage_sum,bureau_credit_type_mortgage_mean,bureau_credit_type_credit_card_sum,bureau_credit_type_credit_card_mean,bureau_credit_type_other_sum,bureau_credit_type_other_mean,bureau_unusual_currency_sum,bureau_unusual_currency_mean,bureau_credit_active_binary
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617,0.0,0.0,...,0.090909,0,0.000000,3,0.272727,0,0.0,0,0.0,6
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000,0.0,NaN,...,0.000000,1,0.166667,2,0.333333,0,0.0,0,0.0,3
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355,0.0,0.0,...,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,1
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060,0.0,0.0,...,0.000000,0,0.000000,3,0.375000,0,0.0,0,0.0,3
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538,0.0,0.0,...,0.125000,0,0.000000,3,0.375000,0,0.0,0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,207190,1,1,1.000000,0.000000,0.00,450000.00,0.000000,0.0,NaN,...,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,0
305807,324956,1,1,1.000000,0.000000,0.00,19800.00,0.000000,0.0,NaN,...,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,0
305808,448157,1,1,1.000000,1.000000,1076809.50,1800000.00,0.598228,0.0,0.0,...,0.000000,1,1.000000,0,0.000000,0,0.0,0,0.0,1
305809,345866,1,1,1.000000,0.000000,0.00,175054.50,0.000000,0.0,NaN,...,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,0


In [23]:
group_object = groupby_SK_ID_CURR['SK_ID_BUREAU'].agg('nunique').reset_index()
group_object.rename(index=str, columns={'SK_ID_BUREAU': 'bureau_query_count'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio,bureau_total_customer_overdue,bureau_overdue_debt_ratio,...,bureau_credit_type_mortgage_sum,bureau_credit_type_mortgage_mean,bureau_credit_type_credit_card_sum,bureau_credit_type_credit_card_mean,bureau_credit_type_other_sum,bureau_credit_type_other_mean,bureau_unusual_currency_sum,bureau_unusual_currency_mean,bureau_credit_active_binary,bureau_query_count
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617,0.0,0.0,...,0,0.000000,3,0.272727,0,0.0,0,0.0,6,11
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000,0.0,NaN,...,1,0.166667,2,0.333333,0,0.0,0,0.0,3,6
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355,0.0,0.0,...,0,0.000000,0,0.000000,0,0.0,0,0.0,1,1
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060,0.0,0.0,...,0,0.000000,3,0.375000,0,0.0,0,0.0,3,8
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538,0.0,0.0,...,0,0.000000,3,0.375000,0,0.0,0,0.0,5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,207190,1,1,1.000000,0.000000,0.00,450000.00,0.000000,0.0,NaN,...,0,0.000000,0,0.000000,0,0.0,0,0.0,0,1
305807,324956,1,1,1.000000,0.000000,0.00,19800.00,0.000000,0.0,NaN,...,0,0.000000,0,0.000000,0,0.0,0,0.0,0,1
305808,448157,1,1,1.000000,1.000000,1076809.50,1800000.00,0.598228,0.0,0.0,...,1,1.000000,0,0.000000,0,0.0,0,0.0,1,1
305809,345866,1,1,1.000000,0.000000,0.00,175054.50,0.000000,0.0,NaN,...,0,0.000000,0,0.000000,0,0.0,0,0.0,0,1


In [24]:
BUREAU_AGGREGATION_RECIPIES = [('CREDIT_TYPE', 'count'),
                               ('CREDIT_ACTIVE', 'size')
                               ]
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_ANNUITY',
                   'AMT_CREDIT_SUM',
                   'AMT_CREDIT_SUM_DEBT',
                   'AMT_CREDIT_SUM_LIMIT',
                   'AMT_CREDIT_SUM_OVERDUE',
                   'AMT_CREDIT_MAX_OVERDUE',
                   'CNT_CREDIT_PROLONG',
                   'CREDIT_DAY_OVERDUE',
                   'DAYS_CREDIT',
                   'DAYS_CREDIT_ENDDATE',
                   'DAYS_CREDIT_UPDATE',
                   'bureau_credit_type_consumer',
                   'bureau_credit_type_car',
                   'bureau_credit_type_credit_card',
                   'bureau_credit_type_mortgage',
                   'bureau_credit_type_other'
                   ]:
        BUREAU_AGGREGATION_RECIPIES.append((select, agg))
BUREAU_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], BUREAU_AGGREGATION_RECIPIES)]


In [27]:
bureau =bureau.sort_values(['SK_ID_CURR', 'DAYS_CREDIT'])
#features_all = pd.DataFrame({'SK_ID_CURR': X['SK_ID_CURR'].unique()})
groupby_aggregate_names = []
number_of_applications = [1, 3, 5]
#number = 1
for number in number_of_applications:
    for groupby_cols, specs in tqdm(BUREAU_AGGREGATION_RECIPIES):
        group_object = bureau.groupby(groupby_cols).tail(number)
        group_object = group_object.groupby(groupby_cols)
        # prev_applications_tail = prev_app_sorted.groupby('SK_ID_CURR').tail(number)
        # tail_groupby = prev_applications_tail.groupby('SK_ID_CURR')
        for select, agg in tqdm(specs):
            groupby_aggregate_name = '{}_{}_{}_{}_{}'.format('_'.join(groupby_cols), agg, select,'last', number)
            features = features.merge(group_object[select]
                                .agg(agg)
                                .reset_index()
                                .rename(index=str,
                                        columns={select: groupby_aggregate_name})
                                [groupby_cols + [groupby_aggregate_name]],
                                on=groupby_cols,
                                how='left')
            groupby_aggregate_names.append(groupby_aggregate_name)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

In [28]:
features

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_average_of_past_loans_per_type,bureau_credit_active_percen,bureau_total_customer_debt,bureau_total_customer_credit,bureau_debt_credit_ratio,bureau_total_customer_overdue,bureau_overdue_debt_ratio,...,SK_ID_CURR_var_CNT_CREDIT_PROLONG_last_5,SK_ID_CURR_var_CREDIT_DAY_OVERDUE_last_5,SK_ID_CURR_var_DAYS_CREDIT_last_5,SK_ID_CURR_var_DAYS_CREDIT_ENDDATE_last_5,SK_ID_CURR_var_DAYS_CREDIT_UPDATE_last_5,SK_ID_CURR_var_bureau_credit_type_consumer_last_5,SK_ID_CURR_var_bureau_credit_type_car_last_5,SK_ID_CURR_var_bureau_credit_type_credit_card_last_5,SK_ID_CURR_var_bureau_credit_type_mortgage_last_5,SK_ID_CURR_var_bureau_credit_type_other_last_5
0,215354,11,3,3.666667,0.545455,284463.18,5973945.30,0.047617,0.0,0.0,...,0.0,0.0,7270.0,1.810594e+08,38.0,0.3,0.0,0.3,0.0,0.0
1,162297,6,3,2.000000,0.500000,0.00,8230386.15,0.000000,0.0,NaN,...,0.0,0.0,328707.2,7.638668e+06,361239.2,0.3,0.0,0.3,0.2,0.0
2,402440,1,1,1.000000,1.000000,76905.00,89910.00,0.855355,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,238881,8,2,4.000000,0.375000,552730.50,1285239.06,0.430060,0.0,0.0,...,0.0,0.0,18682.5,1.146326e+06,5777.7,0.3,0.0,0.3,0.0,0.0
4,222183,8,3,2.666667,0.625000,1185081.84,7158960.00,0.165538,0.0,0.0,...,0.0,0.0,54326.3,6.001167e+05,79039.0,0.3,0.2,0.3,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,207190,1,1,1.000000,0.000000,0.00,450000.00,0.000000,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305807,324956,1,1,1.000000,0.000000,0.00,19800.00,0.000000,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305808,448157,1,1,1.000000,1.000000,1076809.50,1800000.00,0.598228,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305809,345866,1,1,1.000000,0.000000,0.00,175054.50,0.000000,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
features.to_parquet("/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/features_test/bureau.parquet")